In [0]:
#Source: https://www.cloudfronts.com/azure/azure-databricks-how-to-read-csv-file-from-blob-storage-#and-push-the-data-into-a-synapse-sql-pool-table/
from azure.storage.blob import BlobServiceClient
import pandas as pd

import io

import pyspark.sql

storage_account_name = "capstone606storage"

storage_account_access_key = 'U9/wqUO7SoqF1pTv4W9hp7yP1kAc36f4825UX5KsVLbeMfEf/JNN3iglTUGnX/MYjf7FyWTweqoR+ASthrjt4g=='

spark.conf.set('fs.azure.account.key.' + storage_account_name + '.blob.core.windows.net', storage_account_access_key)

blob_container = 'storagecontainer'

filePath = "wasbs://" + blob_container + "@" + storage_account_name + ".blob.core.windows.net/u1008.csv"

empDf = spark.read.format("csv").load(filePath, inferSchema = True, header = True)

empDf.show(10)
# connectionString=”DefaultEndpointsProtocol=https;AccountName=capstone606storage;AccountKey=U9/wqUO7SoqF1pTv4W9hp7yP1kAc36f4825UX5KsVLbeMfEf/JNN3iglTUGnX/MYjf7FyWTweqoR+ASthrjt4g==;EndpointSuffix=core.windows.net”

# empDf.write.jdbc(connectionString,”[dbo].[Employee]”, mode=”append”)

  File "<command-3420688240022827>", line 2
    from azure-storage.blob import BlobServiceClient
              ^
SyntaxError: invalid syntax


In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Capstone606") \
    .config("spark.jars.packages", "com.microsoft.azure:azure-storage:8.6.6") \
    .getOrCreate()

account_name = "capstone606storage"
account_key = "U9/wqUO7SoqF1pTv4W9hp7yP1kAc36f4825UX5KsVLbeMfEf/JNN3iglTUGnX/MYjf7FyWTweqoR+ASthrjt4g=="
container_name = "storagecontainer"
blob_name = "u1008"

df = spark.read.format("com.microsoft.azure.storage.blob") \
    .option("spark.storage.blockManagerTimeoutIntervalMs", "200000") \
    .option("fs.azure.account.key.{0}.blob.core.windows.net".format(account_name), account_key) \
    .option("inferSchema", "true") \
    .option("header", "true") \
    .load("wasbs://{0}@{1}.blob.core.windows.net/{2}/{3}"
          .format(container_name, account_name, blob_name, blob_name+".csv"))

df.show()


---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
<command-3420688240022828> in <cell line: 13>()
     11 blob_name = "u1008"
     12 
---> 13 df = spark.read.format("com.microsoft.azure.storage.blob") \
     14     .option("spark.storage.blockManagerTimeoutIntervalMs", "200000") \
     15     .option("fs.azure.account.key.{0}.blob.core.windows.net".format(account_name), account_key) \

/databricks/spark/python/pyspark/instrumentation_utils.py in wrapper(*args, **kwargs)
     46             start = time.perf_counter()
     47             try:
---> 48                 res = func(*args, **kwargs)
     49                 logger.log_success(
     50                     module_name, class_name, function_name, time.perf_counter() - start, signature

/databricks/spark/python/pyspark/sql/readwriter.py in load(self, path, format, schema, **options)
    175         self.options(**options)
    176

In [0]:
from pyspark.sql import SparkSession
from pyspark.dbutils import DBUtils

# import necessary libraries
from pyspark.sql.types import *
# pyspark functions
from pyspark.sql.functions import *

spark = SparkSession.builder \
    .appName("Capstone606") \
    .getOrCreate()

# Replace with your own account and blob information
account_name = dbutils.secrets.get(scope='capstonescope', key='accountName')
account_key = dbutils.secrets.get(scope='capstonescope', key='key')
container_name = dbutils.secrets.get(scope='capstonescope', key='containerName')

folder_name="clickstreamfiles"
#blob_name = "u1008"

dbutils.fs.mount(
source = "wasbs://{0}@{1}.blob.core.windows.net".format(container_name, account_name),
    mount_point= "/mnt/capstone606",
    extra_configs = {"fs.azure.account.key.{0}.blob.core.windows.net".format(account_name):
                    dbutils.secrets.get(scope='capstonescope', key='key')}
    
)  
    

Out[9]: True

In [0]:
from pyspark.sql.functions import col, countDistinct, max, collect_list, sum

# Create an empty PySpark DataFrame with the required columns
schema = "username STRING, num_of_questions_answered INT, num_of_correct_answers INT, num_of_explanations INT, num_of_lectures INT, devicesUsed STRING, confidence_level INT"
mainDf1 = spark.createDataFrame([], schema)
questionsdf = spark.read.csv("/mnt/capstone606/questionsandpayments/questions.csv",header=True,inferSchema=True)
paymentsdf = spark.read.csv("/mnt/capstone606/questionsandpayments/payments.csv",header=True,inferSchema=True)

# File Schema
file_schema = StructType([
    StructField('timestamp', StringType(), False),
    StructField('action_type', StringType(), True),
    StructField('item_id', StringType(), True),
    StructField('cursor_time', LongType(), True),
    StructField('source', StringType(), True),
    StructField('user_answer', StringType(), True),
    StructField('platform', StringType(), True),
    StructField('user_id', StringType(), True)
])

#schema_1 = "timestamp STRING. action_type STRING, item_id STRING, cursor_time INT, source STRING, user_answer STRING, platform STRING"
for i in dbutils.fs.ls("/mnt/capstone606/clickstreamfiles/"):
    df = spark.read.csv(path= i.path, header=True, schema=file_schema)
    num_of_questions_answered = df.filter(col('action_type') == 'respond').agg(countDistinct('item_id')).collect()[0][0]
    
    #rows with the lastest reponse
    latest_responses = df.filter(col('action_type') == 'respond').select('item_id', 'timestamp',            'user_answer').groupBy('item_id').agg(max('timestamp').alias('timestamp'), max('user_answer').alias('user_answer'))
    latest_responses = latest_responses.sort('item_id')
    # Extracting the questions correct answers
#     tempdf = questionsdf.filter(col('question_id').isin(latest_responses.select('item_id').my_rdd.flatMap(lambda x: x)).alias('alias1')).select('question_id', 'correct_answer').sort('question_id')
    #questions_list = list(latest_responses.select('item_id'))
    #tempdf = questionsdf.filter(col('question_id').isin(questions_list).alias('alias1')).select('question_id', 'correct_answer').sort('question_id')
    questions_id = latest_responses.select('item_id').rdd.flatMap(lambda x: x).collect()
    tempdf = questionsdf.filter(col('question_id').isin(questions_id)).select('question_id', 'correct_answer').sort('question_id')

    tempdf = tempdf.withColumnRenamed('correct_answer', 'correct_answer_temp')
    latest_responses = latest_responses.join(tempdf, latest_responses.item_id == tempdf.question_id).select('item_id', 'timestamp', 'user_answer', 'correct_answer_temp')
    #Comparing the list of responses with correct answers
    num_of_correct_answers = latest_responses.filter(col('user_answer') == col('correct_answer_temp')).count()
    # Explanations and Lecture
    num_of_explanations = df.filter(col('item_id').rlike('e')).agg(countDistinct('item_id')).collect()[0][0]
    num_of_lectures = df.filter(col('item_id').rlike('l')).agg(countDistinct('item_id')).collect()[0][0]
    #additional metric
    # First: Type of devices
    device = df.select('platform').distinct().collect()[0][0]
    #Second: Confidence level by "erase_choice" and adaptive offers
    confidence_level = df.filter(col('action_type') == 'erase_choice').count()
    #Third: tags explored by the users
    # How many paygo and pass
#     payments = paymentsdf.filter(col('payment_item_id').isin(df.filter(col('action_type') == 'pay').select('item_id').rdd.flatMap(lambda x: x).collect())).select('type').rdd.flatMap(lambda x: x).collect()
    # Duration
    #duration = paymentsdf.filter(col('payment_item_id').isin(df.filter(col('action_type') == 'pay').select('item_id'))).agg(sum('duaration')).collect()[0][0]
    #Username
    username = str(i.path.split('/')[-1]).split('.')[0]
    #Putting values in global dataframe
    mainDf1 = mainDf1.union(spark.createDataFrame([(username, num_of_questions_answered, num_of_correct_answers, num_of_explanations, num_of_lectures, device, confidence_level)], schema))
    
    df.unpersist() # to avoid any memory issues

mainDf1.cache()
mainDf1.show(10)  
    

# UNMOUNT
#dbutils.fs.unmount("/mnt/capstone606")

+--------+-------------------------+----------------------+-------------------+---------------+-----------+----------------+
|username|num_of_questions_answered|num_of_correct_answers|num_of_explanations|num_of_lectures|devicesUsed|confidence_level|
+--------+-------------------------+----------------------+-------------------+---------------+-----------+----------------+
| u100043|                      605|                   264|                412|             22|       null|              24|
|  u10019|                     1761|                  1412|               1766|              3|     mobile|             105|
|  u10020|                      169|                   115|                119|              2|       null|              19|
|  u10021|                      136|                    96|                114|              1|       null|               3|
|  u10026|                       60|                    42|                 65|              3|       null|              48|


In [0]:
# OPTIMIZED VERSION 1
from pyspark.sql.functions import countDistinct, max, sum

# Create an empty PySpark DataFrame with the required columns
schema = "username STRING, num_of_questions_answered INT, num_of_correct_answers INT, num_of_explanations INT, num_of_lectures INT, devicesUsed STRING, confidence_level INT"
mainDf1 = spark.createDataFrame([], schema)

questionsdf = spark.read.csv("/mnt/capstone606/questionsandpayments/questions.csv", header=True, inferSchema=True)
paymentsdf = spark.read.csv("/mnt/capstone606/questionsandpayments/payments.csv", header=True, inferSchema=True)

# File Schema
file_schema = "timestamp STRING, action_type STRING, item_id STRING, cursor_time LONG, source STRING, user_answer STRING, platform STRING, user_id STRING"
j=0
for i in dbutils.fs.ls("/mnt/capstone606/clickstreamfiles/"):
    df = spark.read.csv(i.path, header=True, schema=file_schema)

    num_of_questions_answered = df.filter("action_type = 'respond'").agg(countDistinct('item_id')).collect()[0][0]

    # rows with the lastest reponse
    latest_responses = df.filter("action_type = 'respond'") \
                         .groupBy('item_id') \
                         .agg(max('timestamp').alias('timestamp'), max('user_answer').alias('user_answer')) \
                         .select('item_id', 'timestamp', 'user_answer')

    # join with questionsdf to get correct answers
    latest_responses = latest_responses.join(questionsdf, latest_responses.item_id == questionsdf.question_id) \
                                       .select('item_id', 'timestamp', 'user_answer', 'correct_answer') \
                                       .cache()

    # Comparing the list of responses with correct answers
    num_of_correct_answers = latest_responses.filter("user_answer = correct_answer").count()

    # Explanations and Lecture
    num_of_explanations = df.filter("item_id rlike 'e'").agg(countDistinct('item_id')).collect()[0][0]
    num_of_lectures = df.filter("item_id rlike 'l'").agg(countDistinct('item_id')).collect()[0][0]

    # additional metrics
    # First: Type of devices
    device = df.select('platform').distinct().collect()[0][0]

    # Second: Confidence level by "erase_choice" and adaptive offers
    confidence_level = df.filter("action_type = 'erase_choice'").count()
    username = str(i.path.split('/')[-1]).split('.')[0]
    #Putting values in global dataframe
    mainDf1 = mainDf1.union(spark.createDataFrame([(username, num_of_questions_answered, num_of_correct_answers, num_of_explanations, num_of_lectures, device, confidence_level)], schema))
    
    df.unpersist() # to avoid any memory issues
    j+=1
    if(j==1000):
        break

mainDf1.cache()
mainDf1.show(10)  


mainDf1.write.format("delta").mode("append").option("path","/mnt/capstone606/output/mainDf").saveAsTable("default.mainDf1")

+--------+-------------------------+----------------------+-------------------+---------------+-----------+----------------+
|username|num_of_questions_answered|num_of_correct_answers|num_of_explanations|num_of_lectures|devicesUsed|confidence_level|
+--------+-------------------------+----------------------+-------------------+---------------+-----------+----------------+
| u100043|                      605|                   264|                412|             22|       null|              24|
|  u10019|                     1761|                  1412|               1766|              3|     mobile|             105|
|  u10020|                      169|                   115|                119|              2|       null|              19|
|  u10021|                      136|                    96|                114|              1|       null|               3|
|  u10026|                       60|                    42|                 65|              3|       null|              48|


In [0]:
mainDf1.write.parquet("/mnt/capstone606/output/mainDf.parquet")

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("Capstone606") \
    .getOrCreate()
mainDf1 = spark.read.format('delta').load("/mnt/capstone606/output/mainDf")


In [0]:
print(mainDf1.count(),len(mainDf1.columns))

1000 7


In [0]:
from pyspark.sql.functions import col, lit, sum
from pyspark.sql.types import IntegerType
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.dbutils import DBUtils

spark = SparkSession.builder \
    .appName("Capstone606") \
    .getOrCreate()

# Create an empty Spark DataFrame with the same schema as main_df
schema = 'question_id STRING, correct_answer STRING, user_answer STRING, correct_flag INT, question_flag INT'
file_schema = "timestamp STRING, action_type STRING, item_id STRING, cursor_time LONG, source STRING, user_answer STRING, platform STRING, user_id STRING"
questions_df = spark.createDataFrame([], schema)

# Read questions.csv as a Spark DataFrame
questionsdf = spark.read.csv("/mnt/capstone606/questionsandpayments/questions.csv",header=True,inferSchema=True)
j=0
# Loop through files in the EdNet-KT4/KT4 directory
for i in dbutils.fs.ls("/mnt/capstone606/clickstreamfiles/"):
    # Read the file as a Spark DataFrame
    df = spark.read.csv(path=i.path, header=True, schema=file_schema)

    # Filter by action_type == 'respond'
    respond_responses = df.filter("action_type = 'respond'")

    # Join with questionsdf on question_id to get correct_answer
    rr = respond_responses.select('item_id').rdd.flatMap(lambda x: x).collect()
    tempdf = questionsdf.select('question_id', 'correct_answer').\
             filter(col('question_id').isin(rr))
    respond_responses2 = respond_responses.join(tempdf, respond_responses.item_id == tempdf.question_id)

    # Drop unnecessary columns and rename user_answer to match main_df
    respond_responses2 = respond_responses2.drop('item_id', 'timestamp', 'cursor_time', 'source', 'action_type', 'platform').\
                         withColumnRenamed('user_response', 'user_answer')

    # Compute correct_flag and question_flag
    respond_responses2 = respond_responses2.withColumn('correct_flag', 
                             (col('user_answer') == col('correct_answer')).cast(IntegerType())).\
                         withColumn('question_flag', lit(1))

    # Append to main_df
    questions_df = questions_df.union(respond_responses2.select(*questions_df.columns))
    j+=1
    if(j==100):
        break
questions_df.cache()


questions_groupby = questions_df.groupBy('question_id').agg(
    sum('correct_flag').alias('correct_flag_no'), 
    sum('question_flag').alias('question_flag_no')
)
#questions_groupby.show(10)

questions_groupby2 = questions_df.join(questions_groupby, on='question_id', how='inner') \
                                .select([col(c) for c in questions_df.columns] + 
                                        [col('correct_flag_no'), col('question_flag_no')])
questions_groupby2 = questionsdf.join(questions_groupby.select(
    col('question_id'),
    (col('correct_flag_no')/col('question_flag_no')).alias('accuracy')
), on='question_id')

questions_groupby2.cache()
questions_groupby2.show(10)


# questions_groupby2.write.format("delta").mode("append").option("path","/mnt/capstone606/output/tables").saveAsTable("default.questionsTable")

+-----------+---------+--------------+--------------+----+-------------+-------------+-------------------+
|question_id|bundle_id|explanation_id|correct_answer|part|         tags|  deployed_at|           accuracy|
+-----------+---------+--------------+--------------+----+-------------+-------------+-------------------+
|      q1550|    b1450|         e1450|             a|   3|   54;183;182|1514566472641| 0.6666666666666666|
|      q1927|    b1576|         e1576|             b|   3|   57;182;181|1514565378640|                1.0|
|      q4527|    b3059|         e3059|             b|   5|          122|1565337118434|0.38461538461538464|
|      q6915|    b5205|         e5205|             a|   6|           72|1558093775995|                0.5|
|      q4538|    b3070|         e3070|             b|   5|           72|1565337136024|              0.875|
|      q6164|    b4696|         e4696|             a|   5|           94|1564125530908|              0.125|
|        q40|      b40|           e40

In [0]:
questions_groupby2.write.format("delta").mode("append").option("path","/mnt/capstone606/output/tables").saveAsTable("default.questionsTable")

In [0]:
questions_groupby2.write.parquet("/mnt/capstone606/output/questions_groupby2Final.parquet")

# questions_groupby2.write.format("delta").mode("append").option("path","/mnt/capstone606/output/tables").saveAsTable("default.questionsTable")

In [0]:
from pyspark.sql.functions import sum

questions_groupby = questions_df.groupBy('question_id').agg(
    sum('correct_flag').alias('correct_flag'), 
    sum('question_flag').alias('question_flag')
)
questions_groupby.show(10)

+-----------+------------+-------------+
|question_id|correct_flag|question_flag|
+-----------+------------+-------------+
|      q1550|           2|            3|
|      q1927|           3|            3|
|      q4527|          10|           26|
|      q6915|           1|            2|
|       q539|          19|           29|
|        q15|           8|           15|
|      q5064|           8|           10|
|      q6056|           3|            7|
|      q1146|          19|           24|
|      q3831|           7|           11|
+-----------+------------+-------------+
only showing top 10 rows



In [0]:
# # OPTIMIZED CODE
# from pyspark.sql.functions import col, lit, broadcast
# from pyspark.sql.types import IntegerType
# from pyspark.sql import SparkSession

# spark = SparkSession.builder.appName("Capstone606").getOrCreate()

# # Create an empty Spark DataFrame with the same schema as main_df
# schema = 'question_id STRING, correct_answer STRING, user_answer STRING, correct_flag INT, question_flag INT'
# file_schema = "timestamp STRING, action_type STRING, item_id STRING, cursor_time LONG, source STRING, user_answer STRING, platform STRING, user_id STRING"
# questions_df = spark.createDataFrame([], schema)

# # Read questions.csv as a Spark DataFrame and broadcast it
# questionsdf = spark.read.csv("/mnt/capstone606/questionsandpayments/questions.csv",header=True,inferSchema=True)
# questionsdf_broadcast = broadcast(questionsdf)

# # Loop through files in the EdNet-KT4/KT4 directory
# for i in dbutils.fs.ls("/mnt/capstone606/clickstreamfiles/"):
#     # Read the file as a Spark DataFrame
#     df = spark.read.csv(path=i.path, header=True, schema=file_schema)

#     # Filter by action_type == 'respond'
#     respond_responses = df.filter("action_type = 'respond'")

#     # Join with questionsdf on question_id to get correct_answer
#     rr = respond_responses.select('item_id').rdd.flatMap(lambda x: x).collect()
#     tempdf = questionsdf_broadcast.select('question_id', 'correct_answer').\
#              filter(col('question_id').isin(rr))
    
#     respond_responses2 = respond_responses.join(tempdf, respond_responses.item_id == tempdf.question_id)

#     # Drop unnecessary columns and rename user_answer to match main_df
#     respond_responses2 = respond_responses2.drop('item_id', 'timestamp', 'cursor_time', 'source', 'action_type', 'platform').\
#                          withColumnRenamed('user_response', 'user_answer')

#     # Compute correct_flag and question_flag
#     respond_responses2 = respond_responses2.withColumn('correct_flag', 
#                              (col('user_answer') == col('correct_answer')).cast(IntegerType())).\
#                          withColumn('question_flag', lit(1))

#     # Append to questions_df
#     questions_df = questions_df.union(respond_responses2)

#     # Limit the number of files processed to 100
#     if questions_df.count() >= 100:
#         break

# # Cache the questions_df DataFrame
# questions_df.cache()
# questions_df.show()

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
<command-4170612332572212> in <cell line: 18>()
     40 
     41     # Append to questions_df
---> 42     questions_df = questions_df.union(respond_responses2)
     43 
     44     # Limit the number of files processed to 100

/databricks/spark/python/pyspark/instrumentation_utils.py in wrapper(*args, **kwargs)
     46             start = time.perf_counter()
     47             try:
---> 48                 res = func(*args, **kwargs)
     49                 logger.log_success(
     50                     module_name, class_name, function_name, time.perf_counter() - start, signature

/databricks/spark/python/pyspark/sql/dataframe.py in union(self, other)
   2546         Also as standard in SQL, this function resolves columns by position (not by name).
   2547         """
-> 2548         return DataFrame(self._jdf.union(other._jdf), self.

In [0]:
questions_df.show()

+-----------+--------------+-----------+------------+-------------+
|question_id|correct_answer|user_answer|correct_flag|question_flag|
+-----------+--------------+-----------+------------+-------------+
|        q13|             a|          a|           1|            1|
|        q13|             a|          b|           0|            1|
|        q13|             a|          a|           1|            1|
|        q15|             d|          d|           1|            1|
|        q16|             c|          c|           1|            1|
|        q20|             b|          a|           0|            1|
|        q56|             c|          d|           0|            1|
|        q65|             b|          b|           1|            1|
|        q65|             b|          d|           0|            1|
|        q65|             b|          c|           0|            1|
|        q86|             a|          a|           1|            1|
|       q108|             a|          a|        

In [0]:
# UNMOUNT
dbutils.fs.unmount("/mnt/capstone606")

/mnt/capstone606 has been unmounted.
Out[8]: True